In this notebook, tomato classification is performed by using transfer learning from pre trained model - ResNet50
This is the first slice of images into train , validation and test set images.



In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

ImportError: Traceback (most recent call last):
  File "c:\users\dhark\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "c:\users\dhark\appdata\local\programs\python\python38\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\dhark\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\dhark\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_ten

In [ ]:
## Tomato name dictionary 
names = {'0' : 'Kumato' , '1' : 'Beefsteak'  , '2' : 'Tigerella' , '3' :'Roma' , '4' : 'Japanese black trifella' , '5' : 'yellow pear' , '6' : 'sun gold', '7' : 'green zebra' , '8' : 'Cherokee purple' , '9' : 'Oxheart' , '10' : 'blue berries' , '11' : 'San Marzano' , '12' : 'Banana legs' , '13' : 'German orange strawberry' , '14' : 'Super sweet 100'}

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    img_class = []
    for filename in os.listdir(folder):
        img_class.append(int(filename[:2]) -1)
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img , (224,224))
        if img is not None:
            images.append(img)
    return images, img_class

images , image_class = load_images_from_folder("../input/tomato-cultivars")

In [ ]:
type(images) , type(image_class)

In [ ]:
images = np.array(images)
images.shape

In [ ]:
image_class = np.array(image_class)
image_class.shape

In [ ]:
## Using sklearn train_test_split 
train, x_val_test, y_train, y_val_test = train_test_split(images, image_class , stratify = image_class, test_size = 0.25 , random_state = 12)
val, test, y_val, y_test = train_test_split(x_val_test, y_val_test , stratify = y_val_test, test_size = 0.4)

print("train size :{}".format(train.shape))
print("y_train size :{}".format(y_train.shape))
##
print("val size :{}".format(val.shape))
print("y_val size :{}".format(y_val.shape))
##
print("test size :{}".format(test.shape))
print("y_test size :{}".format(y_test.shape))

In [ ]:
## number of classes in train data 
pd.Series(y_train).value_counts()

In [ ]:
## no of classes in validation data 
pd.Series(y_val).value_counts()

In [ ]:
## no of classes in test data 
pd.Series(y_test).value_counts()

In [ ]:
## visualize train set images

plt.figure(figsize=(10, 10))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    ## in order for the image to be displayed in its real color 
    rgb = cv2.cvtColor(train[i], cv2.COLOR_BGR2RGB)
    plt.imshow(rgb, cmap = plt.cm.Spectral)
    #
    label = y_train[i]
    name = names[str(label)]
    plt.title(name)
    plt.axis("off")

In [ ]:
## visualize validation set images

plt.figure(figsize=(10, 10))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    ## in order for the image to be displayed in its real color 
    rgb = cv2.cvtColor(val[i], cv2.COLOR_BGR2RGB)
    plt.imshow(rgb, cmap = plt.cm.Spectral)
    #
    label = y_val[i]
    name = names[str(label)]
    plt.title(name)
    plt.axis("off")

In [ ]:
## visualize test set images

plt.figure(figsize=(10, 10))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    ## in order for the image to be displayed in its real color 
    rgb = cv2.cvtColor(test[i], cv2.COLOR_BGR2RGB)
    plt.imshow(rgb, cmap = plt.cm.Spectral)
    #
    label = y_test[i]
    name = names[str(label)]
    plt.title(name)
    plt.axis("off")

In [ ]:
## pre process the input images as required for the model 
train_processed = tf.keras.applications.resnet.preprocess_input(train)
#
val_processed = tf.keras.applications.resnet.preprocess_input(val)
#
test_processed = tf.keras.applications.resnet.preprocess_input(test)

In [ ]:
# Transfer learning with ResNet . 
base_Net = tf.keras.applications.ResNet50(include_top = False, 
                         weights ='imagenet', 
                         input_shape = (224,224,3), 
                         pooling='avg',
                         )

base_Net.trainable = False
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_Net = tf.keras.models.Sequential()
model_Net.add(base_Net)
model_Net.add(tf.keras.layers.Dense(1024 , activation = 'relu'))
model_Net.add(tf.keras.layers.Dense(15, activation=tf.nn.softmax))

model_Net.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model_Net.summary()

# Training the CNN on the Train data and evaluating it on the val data
%time history = model_Net.fit(train_processed , y_train, validation_data = (val_processed , y_val), epochs = 50, verbose = 0 )

##
test_loss, test_acc = model_Net.evaluate(test_processed , y_test)
print("Test image accuracy :{}".format(test_acc))
##
plt.plot(history.history['accuracy'], '-o' ,label='accuracy')
plt.plot(history.history['val_accuracy'], '-o', label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
##
model_Net.save('ResNet_tomato_classifier')
##
test_predicted = model_Net.predict(test_processed)
##
predicted_label = []
for i in test_predicted:
    label =  np.argmax(i, axis = -1)
    predicted_label.append(label)
    
## classification report 
print(classification_report(y_test, predicted_label, target_names = [v for k,v in names.items()]))